In [1]:
import h2o
import numpy as np

In [2]:
h2o.init(ip="localhost", port=54321)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.202-b08, mixed mode)
  Starting server from C:\Users\melissa.wickers\AppData\Local\Continuum\anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\MELISS~1.WIC\AppData\Local\Temp\tmplwb9ysa2
  JVM stdout: C:\Users\MELISS~1.WIC\AppData\Local\Temp\tmplwb9ysa2\h2o_melissa_wickers_started_from_python.out
  JVM stderr: C:\Users\MELISS~1.WIC\AppData\Local\Temp\tmplwb9ysa2\h2o_melissa_wickers_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_melissa_wickers_s3ngb2
H2O cluster total nodes:,1
H2O cluster free memory:,7.061 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"


In [13]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch

hf = h2o.upload_file("prostate.csv", destination_frame="prostate.csv")

x = hf.columns
y = "CAPSULE"
x.remove(y)


# Split dataset giving the training dataset 75% of the data

train,test,valid = hf.split_frame(ratios=[.7, .15])

# For binary classification, response should be a factor
valid[y] = valid[y].asfactor()
test[y] = test[y].asfactor()
train[y] = train[y].asfactor()

gbm_params1 = {'learn_rate': [0.01, 0.1],
                'max_depth': [3, 5, 9],
                'sample_rate': [0.8, 1.0],
                'col_sample_rate': [0.2, 0.5, 1.0]}

# Train and validate a cartesian grid of GBMs
gbm_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator(nfolds = 10),
                          grid_id='gbm_grid1',
                          hyper_params=gbm_params1)

gbm_grid1.train(x=x, y=y,
                training_frame=train,
                validation_frame=valid,
                ntrees=100,
                seed=1)


Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [14]:
gbm_gridperf1 = gbm_grid1.get_grid(sort_by = 'auc', decreasing = True)
gbm_gridperf1

       col_sample_rate learn_rate max_depth sample_rate            model_ids  \
0                  1.0        0.1         5         0.8   gbm_grid1_model_48   
1                  0.5       0.01         9         1.0  gbm_grid1_model_140   
2                  0.5        0.1         5         1.0  gbm_grid1_model_101   
3                  1.0        0.1         5         1.0   gbm_grid1_model_30   
4                  0.5       0.01         9         0.8   gbm_grid1_model_14   
5                  1.0        0.1         9         0.8   gbm_grid1_model_90   
6                  0.5       0.01         9         0.8   gbm_grid1_model_50   
7                  1.0        0.1         9         1.0   gbm_grid1_model_72   
8                  0.5        0.1         9         1.0   gbm_grid1_model_35   
9                  0.5        0.1         9         1.0   gbm_grid1_model_71   
10                 0.5        0.1         5         1.0   gbm_grid1_model_29   
11                 0.5        0.1       

In [15]:
# Grab the top GBM model, chosen by validation AUC
best_gbm1 = gbm_gridperf1.models[0]

# Now let's evaluate the model performance on a test set
# so we get an honest estimate of top model performance
best_gbm_perf1 = best_gbm1.model_performance(test)

In [16]:
best_gbm_perf1.auc()

1.0